<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Bubble_Dew_Temperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

In [2]:
from dotmap import DotMap
from dotmap import DotMap
import jax
import jax.numpy as jnp
from jax.config import config
from jax.experimental.host_callback import id_print
config.update("jax_enable_x64", True) #JAX default is 32 bit - enable 64 bit - double precision

from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'

from tools.tree_array_transform2 import VSC, Comp, Range, RangeArray
import tools.che as che
R=8.314 # J/(mol K)

In [3]:
p = che.Props(['Methanol', 'Ethanol'])

In [4]:
def bubble_T(c):
    return jnp.sum(c.x * p.Pvap(c.T))-c.P

In [5]:
c=DotMap()
c.x = jnp.array([0.5,0.5])
c.T = Range(300., 273., 400.)
c.P = 101325.
c.T = Range(300., 273.15, 400.)
vsc = VSC(c, bubble_T)
vsc.solve(verbosity=0)
print(vsc.v.T)

[343.76648202]


In [6]:
def dew_T(c):
    return jnp.sum(c.y * c.P / p.Pvap(c.T))-1.

c=DotMap()
c.y = jnp.array([0.5,0.5])
c.P = 101325.
c.T = Range(300., 273.15, 400.)
vsc = VSC(c, dew_T)
vsc.solve(verbosity=0)
print(vsc.v.T)

[345.62285603]


In [7]:
PvapVec = jnp.vectorize(p.Pvap, signature='()->(2)')

In [8]:
def bubble_T_Vec(c):
    return jnp.ravel(jnp.sum(c.x*PvapVec(c.T),-1) - c.P)

In [20]:
x1 = jnp.linspace(0,1,20)
x2 = 1-x1

c = DotMap()
c.x = jnp.stack([x1,x2],1)
c.P = 101325
c.T = RangeArray(jnp.full_like(x1, 300), 273.15, 400.)
vsc = VSC(c, bubble_T_Vec)
vsc.solve(verbosity=0)
bubble_T_array = vsc.v.T

In [23]:
def dew_T_Vec(c):
  return jnp.ravel(jnp.sum(c.y/PvapVec(c.T),-1)*c.P - 1.)

y1 = jnp.linspace(0,1,20)
c=DotMap()
c.y = jnp.stack([y1,1-y1],1)
c.P = 101325.
c.T = RangeArray(jnp.full_like(y1,300), 273.15, 400.)
vsc = VSC(c, dew_T_Vec)
vsc.solve(verbosity=0)
dew_T_array = vsc.v.T

In [24]:
fig = make_subplots(rows=1,cols=1)
fig.add_scatter(x=x1, y=bubble_T_array, name='Liq')
fig.add_scatter(x=y1, y=dew_T_array, name='Vap')
fig.update_xaxes(showline=True, linewidth=1, title_text='x1, y1')
fig.update_yaxes(showline=True, linewidth=1, title_text='Temperature (K)')
fig.update_layout(width=600,height=400, title_text=f'MeOH / EtOH ({c.P:.0f} Pa)', title_x=0.5)

In [25]:
x1 = jnp.linspace(0.,1.,11)
x2 = 1-x1

c = DotMap()
c.x = jnp.stack([x1,x2],1)
c.P = jnp.linspace(100000, 200000, 6).reshape(-1,1)
c.T = RangeArray(jnp.full((c.P.size, x1.size, ), 300.), 273.15, 400.)

vsc = VSC(c, bubble_T_Vec)
vsc.solve(verbosity=0)
bubble_T_array = vsc.v.T

In [27]:
y1 = jnp.linspace(0.,1.,11)
c=DotMap()
c.y = jnp.stack([y1,1-y1],1)
c.P = jnp.linspace(100000, 200000, 6).reshape(-1,1)
c.T = RangeArray(jnp.full((c.P.size, y1.size, ), 300.), 273.15, 400.)
vsc = VSC(c, dew_T_Vec)
vsc.solve(verbosity=0)
dew_T_array = vsc.v.T

In [43]:
fig=make_subplots(rows=1,cols=1)
x,y = jnp.meshgrid(x1, jnp.ravel(c.P))
fig.add_surface(z=bubble_T_array, x=x, y=y, name='Liq',opacity=0.5)
fig.add_surface(z=dew_T_array, x=x, y=y, name='Vap', opacity=0.5)
fig.update_layout(width=800,height=600)
